In [1]:
import pandas as pd
import numpy as np
import collections
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt
import math 

from mozo import num_to_range_categories, merge_with_features, how_many_levels, compare_and_combine, compare_and_combine_2, num_to_range_categories, stratified_sample

In [5]:
total = pd.read_csv('~/pml7/konkurs/output/newest_total_20201031_14_40.h5.csv')
total.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 209 columns):
 #   Column                                              Dtype  
---  ------                                              -----  
 0   breadcrumb                                          object 
 1   created_at                                          object 
 2   price_currency                                      object 
 3   price_details                                       object 
 4   price_value                                         float64
 5   seller_address                                      object 
 6   seller_name                                         object 
 7   seller_type                                         object 
 8   feature_czujniki_parkowania_przednie                bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana        bool   
 10  feature_kurtyny_powietrzne                          bool   
 11  feature_klimatyzacja_dwustrefowa      

In [6]:
total.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki_parkowania_przednie',
       'feature_poduszka_powietrzna_chroniąca_kolana',
       ...
       'seller_miasto', 'encoded_seller_miasto_297',
       'encoded_seller_miasto_perc_40', 'param_moc_norm',
       'param_pojemność_skokowa_norm', 'ogloszenie_rok', 'ogloszenie_miesiac',
       'ogloszenie_dzien_tyg', 'ogloszenie_godzina', 'ogloszenie_dni_wisi'],
      dtype='object', length=209)

In [4]:
train['seller_address'].describe()

count                 106385
unique                 11405
top       Radom, Mazowieckie
freq                     657
Name: seller_address, dtype: object

In [5]:
train['seller_address'].value_counts()

Radom, Mazowieckie                                           657
Lublin, Lubelskie                                            523
Kielce, Świętokrzyskie                                       499
Ostrów Mazowiecka, ostrowski, Mazowieckie                    350
Bydgoszcz, Kujawsko-pomorskie                                346
                                                            ... 
Gronowo Elbląskie, elbląski, Warmińsko-mazurskie               1
Skryta 1 - 63-600 Kępno, kępiński, Wielkopolskie (Polska)      1
Szczepanki, brodnicki, Kujawsko-pomorskie                      1
Bolesławice, bolesławiecki, Dolnośląskie                       1
Kostów, kluczborski, Opolskie                                  1
Name: seller_address, Length: 11405, dtype: int64

In [28]:
total = pd.concat([train, test], ignore_index=True)

In [32]:
def get_miasto(adres):
    import re
    
    if adres is None: return '-1'
    
    try:
        regex = r"\d\d-\d\d\d ([\w\s]*)"
        matches = re.search(regex, adres)
        if matches:
            miasto = matches.groups()[0]
        else:
            miasto = (adres.split(', ')[0])
            miasto = re.sub('\d', '', miasto)

        miasto = re.sub(' +', ' ', miasto).upper().strip()

        return miasto
    except: 
        return '-1'
    
    
        
get_miasto('Gronowo Elbląskie 123, elbląski, Warmińsko-mazurskie')
# get_miasto('Skryta 1 - 63-600   Kępno   gorne  , kępiński, Wielkopolskie (Polska)')

'GRONOWO ELBLĄSKIE'

In [33]:
total['seller_miasto'] = total.apply(lambda row: get_miasto(row['seller_address']), axis=1)
total['seller_miasto'].value_counts()


WARSZAWA       11555
KRAKÓW          4283
ŁÓDŹ            3609
WROCŁAW         3452
POZNAŃ          3174
               ...  
JAMNO              1
WARĘŻYN            1
TRZEBUSKA          1
KOŚCIELISKO        1
STRASZÓW           1
Name: seller_miasto, Length: 5328, dtype: int64

In [46]:
feat = 'seller_miasto'
total["new_{}_cat".format(feat)] = total[feat].factorize()[0]
total["new_{}_cat".format(feat)].value_counts()


17      11555
7        4283
4        3609
229      3452
62       3174
        ...  
3261        1
5182        1
3133        1
5054        1
3641        1
Name: new_seller_miasto_cat, Length: 5328, dtype: int64

In [34]:
def categorial_to_percent_range(df, var_cat, n_cat):
    counts = pd.DataFrame(df[var_cat].value_counts()).reset_index()
    
    def get_level(kod):
        x = counts.loc[counts['index'] == kod, [var_cat]]
        if x.empty:
            return -1
        else: 
            return x.values[0][0]
        
    df['encoded_' + var_cat] = df.apply(lambda row: get_level(row[var_cat]), axis=1)
    print(df['encoded_' + var_cat].value_counts())
    
    num_to_range_categories(df, 'encoded_' + var_cat, n_cat=n_cat) 
    df['percent_range_' + var_cat] = df.apply(lambda row: int(row['range_'+('encoded_' + var_cat)+'_cat'].replace('perc_', '')), axis=1)
    print(total['percent_range_' + var_cat].value_counts() )


In [35]:
categorial_to_percent_range(total, 'seller_miasto', 1 + 4 + 6 + 27)

11555    11555
4283      4283
3609      3609
3452      3452
3174      3174
         ...  
89          89
82          82
79          79
77          77
72          72
Name: encoded_seller_miasto, Length: 297, dtype: int64
perc_97    11555
perc_82     4675
perc_74     4657
perc_03     4583
perc_34     4550
perc_53     4542
perc_05     4496
perc_45     4483
perc_63     4444
perc_16     4348
perc_21     4300
perc_50     4297
perc_92     4283
perc_24     4277
perc_79     4270
perc_08     4263
perc_26     4256
perc_11     4244
perc_66     4218
perc_58     4214
perc_55     4182
perc_71     4155
perc_37     4154
perc_42     4154
perc_39     4145
perc_29     4140
perc_13     4097
perc_18     4055
perc_32     4010
perc_61     3917
perc_47     3813
perc_76     3611
perc_89     3609
perc_00     3526
perc_68     3467
perc_87     3452
perc_84     3174
Name: new_cat_encoded_seller_miasto, dtype: int64
['perc_00', 'perc_03', 'perc_05', 'perc_08', 'perc_11', 'perc_13', 'perc_16', 'perc_18', 'perc_21', '

KeyError: 'range_encoded_seller_miasto_cat'

In [47]:
total.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 203 columns):
 #   Column                                              Dtype  
---  ------                                              -----  
 0   breadcrumb                                          object 
 1   created_at                                          object 
 2   price_currency                                      object 
 3   price_details                                       object 
 4   price_value                                         float64
 5   seller_address                                      object 
 6   seller_name                                         object 
 7   seller_type                                         object 
 8   feature_czujniki_parkowania_przednie                bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana        bool   
 10  feature_kurtyny_powietrzne                          bool   
 11  feature_klimatyzacja_dwustrefowa      

In [48]:
# total.info(verbose=True)
print(total['seller_miasto'].value_counts())
print(total['new_seller_miasto_cat'].value_counts())
print(total['encoded_seller_miasto'].value_counts())
print(total['new_cat_encoded_seller_miasto'].value_counts())

# total['percent_range_seller_miasto' + 'seller_miasto'] = total.apply(lambda row: int(row['range_'+('encoded_' + 'seller_miasto')+'_cat'].replace('perc_', '')), axis=1)

WARSZAWA       11555
KRAKÓW          4283
ŁÓDŹ            3609
WROCŁAW         3452
POZNAŃ          3174
               ...  
JAMNO              1
WARĘŻYN            1
TRZEBUSKA          1
KOŚCIELISKO        1
STRASZÓW           1
Name: seller_miasto, Length: 5328, dtype: int64
17      11555
7        4283
4        3609
229      3452
62       3174
        ...  
3261        1
5182        1
3133        1
5054        1
3641        1
Name: new_seller_miasto_cat, Length: 5328, dtype: int64
11555    11555
4283      4283
3609      3609
3452      3452
3174      3174
         ...  
89          89
82          82
79          79
77          77
72          72
Name: encoded_seller_miasto, Length: 297, dtype: int64
perc_97    11555
perc_82     4675
perc_74     4657
perc_03     4583
perc_34     4550
perc_53     4542
perc_05     4496
perc_45     4483
perc_63     4444
perc_16     4348
perc_21     4300
perc_50     4297
perc_92     4283
perc_24     4277
perc_79     4270
perc_08     4263
perc_26     4256
pe

In [ ]:
'seller_miasto', 'encoded_seller_miasto', 'new_cat_encoded_seller_miasto'

    1 miasto powyżej 1 000 000: Warszawa
    4 miasta od 500 000 do 999 999: Kraków, Łódź, Wrocław, Poznań
    6 miast od 250 000 do 499 999: Gdańsk, Szczecin, Bydgoszcz, Lublin, Białystok, Katowice
    27 miast od 100 000 do 249 999: Gdynia, Częstochowa, Radom, Toruń, Sosnowiec, Kielce, Rzeszów, Gliwice, Zabrze, Olsztyn, Bielsko-Biała, Bytom, Zielona Góra, Rybnik, Ruda Śląska, Opole, Tychy, Gorzów Wielkopolski, Płock, Dąbrowa Górnicza, Elbląg, Wałbrzych, Włocławek, Tarnów, Chorzów, Koszalin, Kalisz

    45 miast od 50 000 do 99 999 (największe w tym przedziale to Legnica, Grudziądz, Jaworzno, Słupsk)

In [51]:
def get_wojewodzctwo(adres):
    if adres is None: return '-1'
    
    try: 
        if adres.lower().find("wielkopolsk") >= 0: return 'Wielkopolskie'
        if adres.lower().find("poznan") >= 0: return 'Wielkopolskie'
        if adres.lower().find("poznań") >= 0: return 'Wielkopolskie'
        if adres.lower().find("kalisz") >= 0: return 'Wielkopolskie'
        
        if adres.lower().find("kujawsk") >= 0: return 'Kujawsko-pomorskie'    
        if adres.lower().find("bydgoszcz") >= 0: return 'Kujawsko-pomorskie'    
        if adres.lower().find("toruń") >= 0: return 'Kujawsko-pomorskie'    
        if adres.lower().find("torun") >= 0: return 'Kujawsko-pomorskie'    
        if adres.lower().find("włocławek") >= 0: return 'Kujawsko-pomorskie'    
        
        if adres.lower().find("małopolsk") >= 0: return 'Małopolskie'
        if adres.lower().find("malopolsk") >= 0: return 'Małopolskie'
        if adres.lower().find("kraków") >= 0: return 'Małopolskie'
        if adres.lower().find("krakow") >= 0: return 'Małopolskie'
        if adres.lower().find("tarnów") >= 0: return 'Małopolskie'
        if adres.lower().find("tarnow") >= 0: return 'Małopolskie'
        
        if adres.lower().find("łódź") >= 0: return 'Łódzkie'
        if adres.lower().find("łódzki") >= 0: return 'Łódzkie'
        if adres.lower().find("lodzki") >= 0: return 'Łódzkie'
        
        if adres.lower().find("dolnośląsk") >= 0: return 'Dolnośląskie'
        if adres.lower().find("dolnoslask") >= 0: return 'Dolnośląskie'
        if adres.lower().find("dolny śląsk") >= 0: return 'Dolnośląskie'
        if adres.lower().find("dolny slask") >= 0: return 'Dolnośląskie'
        if adres.lower().find("jelenia góra") >= 0: return 'Dolnośląskie'
        if adres.lower().find("wroclaw") >= 0: return 'Dolnośląskie'
        if adres.lower().find("wrocław") >= 0: return 'Dolnośląskie'
        if adres.lower().find("wałbrzych") >= 0: return 'Dolnośląskie'
        if adres.lower().find("walbrzych") >= 0: return 'Dolnośląskie'
        
        if adres.lower().find("lubels") >= 0: return 'Lubelskie'
        if adres.lower().find("lublin") >= 0: return 'Lubelskie'
        
        if adres.lower().find("lubusk") >= 0: return 'Lubuskie'
        if adres.lower().find("zielona g") >= 0: return 'Lubuskie'
        if adres.lower().find("gorzów") >= 0: return 'Lubuskie'
        if adres.lower().find("gorzow") >= 0: return 'Lubuskie'
        
        if adres.lower().find("mazow") >= 0: return 'Mazowieckie'
        if adres.lower().find("warszawa") >= 0: return 'Mazowieckie'
        if adres.lower().find("radom") >= 0: return 'Mazowieckie'
        if adres.lower().find("płock") >= 0: return 'Mazowieckie'
        if adres.lower().find("plock") >= 0: return 'Mazowieckie'
        
        if adres.lower().find("opole") >= 0: return 'Opolskie'
        if adres.lower().find("opolskie") >= 0: return 'Opolskie'
        if adres.lower().find("kędzierzyn") >= 0: return 'Opolskie'
        if adres.lower().find("kedzierzyn") >= 0: return 'Opolskie'
        
        if adres.lower().find("podlask") >= 0: return 'Podlaskie'
        if adres.lower().find("białystok") >= 0: return 'Podlaskie'
        
        if adres.lower().find("zachodniopomorsk") >= 0: return 'Zachodniopomorskie'
        if adres.lower().find("szczecin") >= 0: return 'Zachodniopomorskie'
        if adres.lower().find("koszalin") >= 0: return 'Zachodniopomorskie'
        
        if adres.lower().find("pomorskie") >= 0: return 'Pomorskie'
        if adres.lower().find("gdańsk") >= 0: return 'Pomorskie'
        if adres.lower().find("gdynia") >= 0: return 'Pomorskie'
        
        if adres.lower().find("slaskie") >= 0: return 'Śląskie'
        if adres.lower().find("śląskie") >= 0: return 'Śląskie'
        if adres.lower().find("śląsk") >= 0: return 'Śląskie'
        if adres.lower().find("gliwice") >= 0: return 'Śląskie'
        if adres.lower().find("częstochowa") >= 0: return 'Śląskie'
        if adres.lower().find("katowice") >= 0: return 'Śląskie'
        if adres.lower().find("mikołów") >= 0: return 'Śląskie'
        if adres.lower().find("zawiercie") >= 0: return 'Śląskie'
        if adres.lower().find("sosnowiec") >= 0: return 'Śląskie'
        if adres.lower().find("zabrze") >= 0: return 'Śląskie'
        if adres.lower().find("bielsko") >= 0: return 'Śląskie'
        if adres.lower().find("bytom") >= 0: return 'Śląskie'
        if adres.lower().find("rybnik") >= 0: return 'Śląskie'
        if adres.lower().find("ruda s") >= 0: return 'Śląskie'
        if adres.lower().find("ruda ś") >= 0: return 'Śląskie'
        if adres.lower().find("dąbrowa") >= 0: return 'Śląskie'
        if adres.lower().find("dabrowa g") >= 0: return 'Śląskie'
        if adres.lower().find("chorzów") >= 0: return 'Śląskie'
       
        if adres.lower().find("podkarpac") >= 0: return 'Podkarpackie'
        if adres.lower().find("rzeszów") >= 0: return 'Podkarpackie'
        if adres.lower().find("rzeszow") >= 0: return 'Podkarpackie'
                
        if adres.lower().find("świętokrzyskie") >= 0: return 'Świętokrzyskie'
        if adres.lower().find("swietokrzyskie") >= 0: return 'Świętokrzyskie'
        if adres.lower().find("kielce") >= 0: return 'Świętokrzyskie'
        
        if adres.lower().find("warmi") >= 0: return 'Warmińsko-Mazurskie'
        if adres.lower().find("mazur") >= 0: return 'Warmińsko-Mazurskie'
        if adres.lower().find("olsztyn") >= 0: return 'Warmińsko-Mazurskie'
        if adres.lower().find("elbląg") >= 0: return 'Warmińsko-Mazurskie'
        if adres.lower().find("elblag") >= 0: return 'Warmińsko-Mazurskie'
        
    except: 
#         print(adres)
        return '-1'

    return '-1'
        
get_wojewodzctwo('małopolsk')
total['seller_wojewodzctwo'] = total.apply(lambda row: get_wojewodzctwo(row['seller_address']), axis=1)
total['seller_wojewodzctwo'].value_counts()


Mazowieckie            28339
Wielkopolskie          23671
Śląskie                18390
Małopolskie            16104
Dolnośląskie           11977
Łódzkie                 9687
Pomorskie               9669
Lubelskie               7225
Kujawsko-pomorskie      6937
Zachodniopomorskie      5608
Podkarpackie            4886
Świętokrzyskie          4763
Warmińsko-Mazurskie     3757
Lubuskie                3116
Podlaskie               2926
Opolskie                2725
-1                       836
Name: seller_wojewodzctwo, dtype: int64

In [53]:
feat = 'seller_wojewodzctwo'
total["new_{}_cat".format(feat)] = total[feat].factorize()[0]
print(total["new_{}_cat".format(feat)].value_counts())


6     28339
2     23671
8     18390
3     16104
5     11977
1      9687
10     9669
4      7225
12     6937
13     5608
0      4886
11     4763
9      3757
15     3116
14     2926
7      2725
16      836
Name: new_seller_wojewodzctwo_cat, dtype: int64


In [41]:
filter_col = ['car_id', 
    'seller_wojewodzctwo', 'encoded_seller_wojewodzctwo', 'percent_range_seller_wojewodzctwo'
]

In [42]:
total.loc[~total['price_value'].isnull(), filter_col].to_csv('output/ft_new_param_4_train.csv', index=False) 
total.loc[total['price_value'].isnull(), filter_col].to_csv('output/ft_new_param_4_test.csv', index=False)

categorial_to_percent_range(total, 'seller_wojewodzctwo', 16 + 1)

In [65]:
total = total.drop('new_seller_miasto_cat', axis=1)
total = total.rename(columns={"encoded_seller_miasto": "encoded_seller_miasto_297", 
                      "new_cat_encoded_seller_miasto": "encoded_seller_miasto_perc_40"}, errors="raise")

In [66]:
today = datetime.today() 
cvs_file_name = f'newest_total_{today:%Y%m%d_%H_%M.h5}'
total.to_csv('~/pml7/konkurs/output/' + cvs_file_name + '.csv', index=False) 
total.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 202 columns):
 #   Column                                              Dtype  
---  ------                                              -----  
 0   breadcrumb                                          object 
 1   created_at                                          object 
 2   price_currency                                      object 
 3   price_details                                       object 
 4   price_value                                         float64
 5   seller_address                                      object 
 6   seller_name                                         object 
 7   seller_type                                         object 
 8   feature_czujniki_parkowania_przednie                bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana        bool   
 10  feature_kurtyny_powietrzne                          bool   
 11  feature_klimatyzacja_dwustrefowa      

In [61]:
total['new_cat_encoded_seller_miasto'].value_counts()


perc_97    11555
perc_82     4675
perc_74     4657
perc_03     4583
perc_34     4550
perc_53     4542
perc_05     4496
perc_45     4483
perc_63     4444
perc_16     4348
perc_21     4300
perc_50     4297
perc_92     4283
perc_24     4277
perc_79     4270
perc_08     4263
perc_26     4256
perc_11     4244
perc_66     4218
perc_58     4214
perc_55     4182
perc_71     4155
perc_37     4154
perc_42     4154
perc_39     4145
perc_29     4140
perc_13     4097
perc_18     4055
perc_32     4010
perc_61     3917
perc_47     3813
perc_76     3611
perc_89     3609
perc_00     3526
perc_68     3467
perc_87     3452
perc_84     3174
Name: new_cat_encoded_seller_miasto, dtype: int64

In [71]:
total['param_moc_norm'] = np.log( total['param_moc'].astype(np.int) )
total['param_pojemność_skokowa_norm'] = np.log( total['param_pojemność_skokowa'].astype(np.int) )
total['param_moc_norm'].describe()
total['param_pojemność_skokowa_norm'].describe()

count    152962.000000
mean          7.494638
std           0.434347
min           0.000000
25%           7.352441
50%           7.547502
75%           7.600902
max          14.914196
Name: param_pojemność_skokowa_norm, dtype: float64

In [72]:
today = datetime.today() 
cvs_file_name = f'newest_total_{today:%Y%m%d_%H_%M.h5}'
total.to_csv('~/pml7/konkurs/output/' + cvs_file_name + '.csv', index=False) 
total.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 204 columns):
 #   Column                                              Dtype  
---  ------                                              -----  
 0   breadcrumb                                          object 
 1   created_at                                          object 
 2   price_currency                                      object 
 3   price_details                                       object 
 4   price_value                                         float64
 5   seller_address                                      object 
 6   seller_name                                         object 
 7   seller_type                                         object 
 8   feature_czujniki_parkowania_przednie                bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana        bool   
 10  feature_kurtyny_powietrzne                          bool   
 11  feature_klimatyzacja_dwustrefowa      

In [79]:
total['created_at'].describe()
total['created_at'].value_counts()

12:13, 2 marca 2018      191
22:19, 28 lutego 2018    168
12:14, 2 marca 2018      164
12:17, 2 marca 2018      162
22:17, 28 lutego 2018    158
                        ... 
18:30, 15 lutego 2018      1
09:44, 23 lutego 2018      1
14:20, 10 lutego 2018      1
10:48, 26 lutego 2018      1
11:16, 22 lutego 2018      1
Name: created_at, Length: 11959, dtype: int64

In [80]:
def get_date(pl_date_string):
    if pl_date_string is not None and len(str(pl_date_string)) > 12:
        hour, date = pl_date_string.split(', ')
        day, month, year = date.split(' ')
        return day, month, year
    return 'NA'

total.apply(lambda row: get_date(row['created_at']), axis=1)

0           (5, marca, 2018)
1           (5, marca, 2018)
2           (4, marca, 2018)
3           (3, marca, 2018)
4           (3, marca, 2018)
                 ...        
160611    (22, lutego, 2018)
160612      (5, marca, 2018)
160613      (3, marca, 2018)
160614      (1, marca, 2018)
160615      (4, marca, 2018)
Length: 160616, dtype: object

In [88]:
pd.to_datetime((pd.to_datetime('2021-12-05 17:40:10').value / 10**9) * 10**9).year

2021

In [120]:
total['created_at_unix_na_mean'].describe()

count    1.606160e+05
mean     1.520016e+09
std      2.231692e+05
min      1.513965e+09
25%      1.519898e+09
50%      1.520038e+09
75%      1.520181e+09
max      1.520602e+09
Name: created_at_unix_na_mean, dtype: float64

In [119]:
min(total['created_at_unix'])

0.0

In [102]:
(pd.to_datetime(1520602200.0 * 10**9) - pd.to_datetime(1513965240.0 * 10**9)).total_seconds() / (60 * 60 * 24)

76.81666666666666

In [108]:
def get_date(unix):
    if unix == (pd.to_datetime('1970-01-01 00:00:00').value / 10**9) : return -1
    
    pandastime = pd.to_datetime(unix * 10**9)
    
    dni_wisi = (pd.to_datetime(1520602200.0 * 10**9) - pd.to_datetime(unix * 10**9)).total_seconds() / (60 * 60 * 24)
    
    return pandastime.year, pandastime.month, pandastime.dayofweek, pandastime.hour, dni_wisi
    
    return -1, -1, -1, -1, -1

get_date(pd.to_datetime('2020-10-31 17:40:10').value / 10**9)

(2020, 10, 5, 17, -967.1737268518518)

In [128]:
total['ogloszenie_rok'] = -1
total['ogloszenie_miesiac'] = -1
total['ogloszenie_dzien_tyg'] = -1
total['ogloszenie_godzina'] = -1
total['ogloszenie_dni_wisi'] = -1

# total[['ogloszenie_rok', 'ogloszenie_miesiac', 'ogloszenie_dzien_tyg', 'ogloszenie_godzina', 
#        'ogloszenie_dni_wisi']] = -1, -1, -1, -1, -1

total[['ogloszenie_rok', 'ogloszenie_miesiac', 'ogloszenie_dzien_tyg', 'ogloszenie_godzina', 
       'ogloszenie_dni_wisi']] = total.apply(lambda row: pd.Series(get_date(row['created_at_unix'])), axis=1)
# total = total.drop('ogloszenie_dzien_dni_wisi', axis=1)

total.loc[total['ogloszenie_rok'] == -1, ['ogloszenie_rok', 'ogloszenie_miesiac', 'ogloszenie_dzien_tyg', 'ogloszenie_godzina', 
       'ogloszenie_dni_wisi']] = -1, -1, -1, -1, -1

print(total['ogloszenie_rok'].value_counts())
print(total['ogloszenie_miesiac'].value_counts())
print(total['ogloszenie_dzien_tyg'].value_counts())
print(total['ogloszenie_godzina'].value_counts())
print(total['ogloszenie_dni_wisi'].value_counts())


 2018.0    160452
-1.0          161
 2017.0         3
Name: ogloszenie_rok, dtype: int64
 3.0     122773
 2.0      37655
-1.0        161
 1.0         24
 12.0         3
Name: ogloszenie_miesiac, dtype: int64
 0.0    32190
 6.0    27216
 5.0    25216
 4.0    25056
 3.0    19951
 2.0    16375
 1.0    14451
-1.0      161
Name: ogloszenie_dzien_tyg, dtype: int64
 12.0    12899
 15.0    12498
 16.0    12388
 13.0    12343
 14.0    11950
 11.0    11628
 10.0    11362
 17.0    10795
 9.0      9657
 18.0     9533
 21.0     8321
 20.0     7911
 19.0     7898
 22.0     6831
 8.0      5375
 23.0     3163
 7.0      2421
 0.0      1386
 6.0       903
 1.0       495
 2.0       282
 5.0       165
-1.0       161
 3.0       153
 4.0        98
Name: ogloszenie_godzina, dtype: int64
 7.053472     191
 8.632639     168
 7.052778     164
 7.050694     162
-1.000000     161
             ... 
 14.927778      1
 23.277083      1
 11.946528      1
 17.961111      1
 14.000694      1
Name: ogloszenie_dni_wisi, 

In [129]:
total.info(verbose=True)

today = datetime.today() 
cvs_file_name = f'newest_total_{today:%Y%m%d_%H_%M.h5}'
total.to_csv('~/pml7/konkurs/output/' + cvs_file_name + '.csv', index=False) 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 209 columns):
 #   Column                                              Dtype  
---  ------                                              -----  
 0   breadcrumb                                          object 
 1   created_at                                          object 
 2   price_currency                                      object 
 3   price_details                                       object 
 4   price_value                                         float64
 5   seller_address                                      object 
 6   seller_name                                         object 
 7   seller_type                                         object 
 8   feature_czujniki_parkowania_przednie                bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana        bool   
 10  feature_kurtyny_powietrzne                          bool   
 11  feature_klimatyzacja_dwustrefowa      

In [7]:
total['param_pierwsza_rejestracja_ten_rok'] = total.apply(lambda row: -1 if row['created_at_unix'] == 0.0 or row['param_pierwsza_rejestracja_date'] == 0.0
    else 1 if pd.to_datetime(row["param_pierwsza_rejestracja_date"] * 10**9).year == pd.to_datetime(row["created_at_unix"] * 10**9).year else 0,
    axis=1)
print(total['param_pierwsza_rejestracja_ten_rok'].value_counts())

total['param_pierwsza_rejestracja_poprzedni_rok'] = total.apply(lambda row: -1 if row['created_at_unix'] == 0.0 or row['param_pierwsza_rejestracja_date'] == 0.0
    else 1 if pd.to_datetime(row["param_pierwsza_rejestracja_date"] * 10**9).year + 1 == pd.to_datetime(row["created_at_unix"] * 10**9).year else 0,
    axis=1)
print(total['param_pierwsza_rejestracja_ten_rok'].value_counts())

total['param_pierwsza_rejestracja_10_lat'] = total.apply(lambda row: -1 if row['created_at_unix'] == 0.0 or row['param_pierwsza_rejestracja_date'] == 0.0
    else 1 if pd.to_datetime(row["param_pierwsza_rejestracja_date"] * 10**9).year +10 == pd.to_datetime(row["created_at_unix"] * 10**9).year else 0,
    axis=1)
print(total['param_pierwsza_rejestracja_ten_rok'].value_counts())



 0    160069
 1       386
-1       161
Name: param_pierwsza_rejestracja_ten_rok, dtype: int64
 0    160069
 1       386
-1       161
Name: param_pierwsza_rejestracja_ten_rok, dtype: int64
 0    160069
 1       386
-1       161
Name: param_pierwsza_rejestracja_ten_rok, dtype: int64


In [9]:
print(total['param_rok_produkcji'].value_counts())


2017    17673
2007    11310
2008    11066
2006    10390
2009     9458
        ...  
1933        1
1932        1
1931        1
1930        1
1942        1
Name: param_rok_produkcji, Length: 82, dtype: int64


In [10]:
total['param_rok_produkcji_ten_rok'] = total.apply(lambda row: -1 if row['created_at_unix'] == 0.0 or row['param_rok_produkcji'] == -1
    else 1 if int(row["param_rok_produkcji"]) == pd.to_datetime(row["created_at_unix"] * 10**9).year else 0,
    axis=1)
print(total['param_rok_produkcji_ten_rok'].value_counts())

total['param_rok_produkcji_poprzedni_rok'] = total.apply(lambda row: -1 if row['created_at_unix'] == 0.0 or row['param_rok_produkcji'] == -1
    else 1 if int(row["param_rok_produkcji"]) + 1 == pd.to_datetime(row["created_at_unix"] * 10**9).year else 0,
    axis=1)
print(total['param_rok_produkcji_ten_rok'].value_counts())

total['param_rok_produkcji_10_lat'] = total.apply(lambda row: -1 if row['created_at_unix'] == 0.0 or row['param_rok_produkcji'] == -1
    else 1 if int(row["param_rok_produkcji"]) +10 == pd.to_datetime(row["created_at_unix"] * 10**9).year else 0,
    axis=1)
print(total['param_rok_produkcji_ten_rok'].value_counts())



 0    156912
 1      3543
-1       161
Name: param_rok_produkcji_ten_rok, dtype: int64
 0    156912
 1      3543
-1       161
Name: param_rok_produkcji_ten_rok, dtype: int64
 0    156912
 1      3543
-1       161
Name: param_rok_produkcji_ten_rok, dtype: int64


In [11]:
total['param_ile_lat_temu_produkcja'] = total.apply(lambda row: -1 if row['created_at_unix'] == 0.0 
    else pd.to_datetime(row["created_at_unix"] * 10**9).year - row["param_rok_produkcji"],
    axis=1)
print(total['param_ile_lat_temu_produkcja'].value_counts())


1     17672
11    11310
10    11067
12    10389
9      9458
      ...  
82        1
85        1
86        1
87        1
88        1
Name: param_ile_lat_temu_produkcja, Length: 82, dtype: int64


In [ ]:
total.info(verbose=True)

today = datetime.today() 
cvs_file_name = f'newest_total_{today:%Y%m%d_%H_%M.h5}'
total.to_csv('~/pml7/konkurs/output/' + cvs_file_name + '.csv', index=False) 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 216 columns):
 #   Column                                              Dtype  
---  ------                                              -----  
 0   breadcrumb                                          object 
 1   created_at                                          object 
 2   price_currency                                      object 
 3   price_details                                       object 
 4   price_value                                         float64
 5   seller_address                                      object 
 6   seller_name                                         object 
 7   seller_type                                         object 
 8   feature_czujniki_parkowania_przednie                bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana        bool   
 10  feature_kurtyny_powietrzne                          bool   
 11  feature_klimatyzacja_dwustrefowa      